Reference: https://yangkky.github.io/2019/07/08/distributed-pytorch-tutorial.html

Two steps for training model on a single GPU on a single node:
1. set gpu number that we want to use
2. send each params that we want to compute with to GPU

In [1]:
from datetime import datetime

import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.distributed as dist

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

def main(gpu, args):
    torch.manual_seed(0)
    model = ConvNet()
    
    # step 1: set gpu number that we want to use
    torch.cuda.set_device(gpu)
    
    # step 2: send each params that we want to compute with to GPU
    model.cuda(gpu)
    
    batch_size = 100
    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(gpu)
    optimizer = torch.optim.SGD(model.parameters(), 1e-4)
    
    # Data loading code
    train_dataset = torchvision.datasets.MNIST(root='./data',
                                               train=True,
                                               transform=transforms.ToTensor(),
                                               download=True)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=0,
                                               pin_memory=True)

    start = datetime.now()
    total_step = len(train_loader)
    for epoch in range(args.epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i + 1) % 100 == 0 and gpu == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch + 1, 
                    args.epochs, 
                    i + 1, 
                    total_step,
                    loss.item())
                   )
    if gpu == 0:
        print("Training complete in: " + str(datetime.now() - start))

class config:
    n = 1
    g = 1
    nr = 0
    epochs = 1

main(0, config())

Epoch [1/1], Step [100/600], Loss: 2.1136
Epoch [1/1], Step [200/600], Loss: 1.9958
Epoch [1/1], Step [300/600], Loss: 1.8824
Epoch [1/1], Step [400/600], Loss: 1.7341
Epoch [1/1], Step [500/600], Loss: 1.7331
Epoch [1/1], Step [600/600], Loss: 1.5436
Training complete in: 0:00:13.639372
